## Copy T1-space maps and DARTEL imports to a single folder

Maybe DARTEL imports are not necessary and can be used directly from the origin folder.

- Copy T1-space DWI images to a single folder
- Make sure they are already gunzipped
- Make sure that c1 and dwi do match correctly (remove headers from c1 if necessary)

In [35]:
from glob import glob
import os
import os.path as osp
ind = '/home/grg/data/AmylStaging'
oud = '/tmp/dartel_FA'

dartel_dir = '/home/grg/data/AmylStaging/dartel'

fa_maps = sorted([e for e in glob(osp.join(ind, '*', 'DWI', '*_FA_t1space*')) if e.endswith('.nii') or e.endswith('.nii.gz')])
md_maps = sorted([e for e in glob(osp.join(ind, '*', 'DWI', '*_MD_t1space*')) if e.endswith('.nii') or e.endswith('.nii.gz')])
l1_maps = sorted([e for e in glob(osp.join(ind, '*', 'DWI', '*_L1_t1space*')) if e.endswith('.nii') or e.endswith('.nii.gz')])
rd_maps = sorted([e for e in glob(osp.join(ind, '*', 'DWI', '*_RD_t1space*')) if e.endswith('.nii') or e.endswith('.nii.gz')])
rc1_maps = sorted([e for e in glob(osp.join(ind, '*', 'T1', '*_rc1*')) if e.endswith('.nii') or e.endswith('.nii.gz')])
c1_maps = sorted([e for e in glob(osp.join(ind, '*', 'T1', '*_c1*')) if e.endswith('.nii') or e.endswith('.nii.gz')])

print('%s FA maps'%len(fa_maps))
print('%s MD maps'%len(md_maps))
print('%s L1 maps'%len(l1_maps))
print('%s RD maps'%len(rd_maps))
print('%s rc1 maps'%len(rc1_maps))
print('%s c1 maps'%len(c1_maps))

cmds = []
for maps in [fa_maps, md_maps, l1_maps, rd_maps]:
  for each in maps:
    bn = osp.basename(each)
    fp = osp.join(oud, 'native', bn)
    cmd = 'cp %s %s; gunzip %s'%(each, fp, fp)
    #cmds.append(cmd)
    
for i, j in zip(c1_maps, rc1_maps):
    bn = osp.basename(i)
    fp = osp.join(oud, 'c1', bn)
    cmd = 'cp %s %s'%(i, fp)
    #cmds.append(cmd)
    bn = osp.basename(j)
    fp = osp.join(oud, 'rc1', bn)
    cmd = 'cp %s %s'%(j, fp)
    #cmds.append(cmd)
    
# remove headers from native and c1
for e in glob(osp.join(oud, 'native', '*nii')):
    cmd = 'python /home/grg/git/alfa/remove_headers.py %s --replace'%e
    cmds.append(cmd)

fp1 = '/tmp/preparation_dartelnorm.sh'
w = open(fp1, 'w')
for e in cmds:    
    w.write('%s\n'%e)
w.close()

114 FA maps
114 MD maps
114 L1 maps
114 RD maps
114 rc1 maps
114 c1 maps


In [37]:
len(cmds)

456

### First coregister T1-space DWI images

In [26]:
import os
cmd_coreg_dwi = 'python /home/grg/git/alfa/spm/CoregDWI.py'
cmd_dartelnorm2mni = 'python /home/grg/git/alfa/spm/DARTELNorm2MNI.py'
cmd_removeheaders = 'python /home/grg/git/alfa/remove_headers.py'
cmd_coreg_mni = 'python /home/grg/git/alfa/spm/CoregMNI.py'

c1_fp = osp.join(oud, 'c1')
rc1_fp = osp.join(oud, 'rc1')
dwi_t1space_fp = osp.join(oud, 'native')
subjects_fp = '/tmp/amyloid_subjects.json'
realigned_dwi_fp = osp.join(oud, 'realigned_FA')
cmd = '%s %s %s %s %s %s "_FA"'%(cmd_coreg_dwi, c1_fp, rc1_fp, dwi_t1space_fp, subjects_fp, realigned_dwi_fp)
print cmd

python /home/grg/git/alfa/spm/CoregDWI.py /tmp/dartel_FA/c1 /tmp/dartel_FA/rc1 /tmp/dartel_FA/native /tmp/amyloid_subjects.json /tmp/dartel_FA/realigned_FA "_FA"


- Move realigned DWI images (from realigned_FA/*/*) to same folder

In [34]:
cmd = 'cp %s/*/*/r*t1space.nii %s'%(realigned_dwi_fp, realigned_dwi_fp)
#os.system(cmd)
print cmd

cp /tmp/dartel_FA/realigned_FA/*/*/r*t1space.nii /tmp/dartel_FA/realigned_FA


### DARTELNorm2MNI

In [39]:
ff_fp = '/home/grg/data/AmylStaging/dartel/'
subjects_fp = '/tmp/amyloid_subjects_few.json'
tpl_fp = '/home/grg/data/AmylStaging/dartel/Template_6.nii'
fp = 'FA_t1space'
cmd = '%s %s %s %s %s %s %s'%(cmd_dartelnorm2mni, ff_fp, realigned_dwi_fp, rc1_fp, subjects_fp, tpl_fp, fp)
print cmd

python /home/grg/git/alfa/spm/DARTELNorm2MNI.py /home/grg/data/AmylStaging/dartel/ /tmp/dartel_FA/realigned_FA /tmp/dartel_FA/rc1 /tmp/amyloid_subjects_few.json /home/grg/data/AmylStaging/dartel/Template_6.nii FA_t1space


- Move the warped DWI images (from `[rc1_fp]/[DARTELNorm2MNI]` folder) to the same single folder `[warped_fp]`


### Coregister with MNI template

In [11]:
cmd = '%s %s .'%(cmd_coreg_mni, warped_fp)
print cmd

python /home/grg/git/alfa/spm/CoregMNI.py /tmp/dartel/warped_fa .


### Remove headers from final DWI images

In [10]:
warped_fp = '/tmp/dartel/warped_fa'
cmd = '%s %s/*'%(cmd_removeheaders, warped_fp)
print cmd

python /home/grg/git/alfa/remove_headers.py /tmp/dartel/warped_fa/*
